In [2]:
import requests
from pypac import PACSession
import pandas as pd
import json
from pandas import json_normalize
import datetime

In [3]:
curr_time = datetime.datetime.now()
hq_csv =  "../Stock_data/hangqing/" + "hangqing_A_" + str(curr_time.date()) + ".csv"
df1 = pd.read_csv(hq_csv,dtype={'f12':str})
df1[['f12','f14', 'f13']].head(10)
df1.set_index('f12', inplace=True)
#df1.loc['300820', 'f13']
#df1.index[:10]

filed_num = 0

for stock_code in df1.index[-40:-20]:
    
    market_id = df1.loc[stock_code, 'f13'] #上证是1，深证是0
    if str(market_id) == '0':
        stock_id = stock_code + '2' #上证是1，深证是2
    else:
        stock_id = stock_code + '1' #上证是1，深证是2
    
    stock_name = df1.loc[stock_code, 'f14']
    csv_file_name = "../Stock_data/fenshi/" + "fenshi_" + stock_code + "_" + stock_name + "_" + str(curr_time.date()) + ".csv"

    for page_num in range(50):
        try:
            cookies = {
                'em_hq_fls': 'js',
                'pgv_pvi': '9744040960',
                '_qddaz': 'QD.efn5wx.2y13rz.kbiype8t',
                'intellpositionL': '720.95px',
                'cowminicookie': 'true',
                'emshistory': '^%^5B^%^22600519^%^22^%^5D',
                'intellpositionT': '755px',
                'qgqp_b_id': '3b5494631b0d78184891d0de53c4b3d9',
                'st_si': '81244739108722',
                'st_asi': 'delete',
                '_sm_au_c': 'kLQABAOhavxwaGMtlop7idfc5VbaufLUcxaJVNQw0aBAgAAAAXYqAxNyvGhuw4qcGTDpRh0vz6imEFA2Tbt8n3TUV7ks=',
                'st_pvi': '20795103616993',
                'st_sp': '2020-06-17^%^2014^%^3A02^%^3A47',
                'st_inirUrl': 'https^%^3A^%^2F^%^2Fcn.bing.com^%^2F',
                'st_sn': '26',
                'st_psi': '20200629111105268-0-8277370083',
                'HAList': 'a-sh-601865-^%^u798F^%^u83B1^%^u7279^%^2Ca-sz-002813-^%^u8DEF^%^u7545^%^u79D1^%^u6280^%^2Ca-sh-600956-N^%^u65B0^%^u5929^%^2Ca-sh-600519-^%^u8D35^%^u5DDE^%^u8305^%^u53F0^%^2Ca-sz-000858-^%^u4E94^%^20^%^u7CAE^%^20^%^u6DB2^%^2Ca-sz-002129-^%^u4E2D^%^u73AF^%^u80A1^%^u4EFD^%^2Ca-sh-603365-^%^u6C34^%^u661F^%^u5BB6^%^u7EBA^%^2Ca-sh-600918-^%^u4E2D^%^u6CF0^%^u8BC1^%^u5238',
            }

            headers = {
                'Proxy-Connection': 'keep-alive',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
                'Accept': '*/*',
                'Referer': 'http://quote.eastmoney.com/f1.html?code=601865^&market=1',
                'Accept-Language': 'en-US,en;q=0.9',
            }

            # params = (
            #     ('pagesize', '144^'),
            #     ('ut', '7eea3edcaed734bea9cbfc24409ed989^'),
            #     ('dpt', 'wzfscj^'),
            #     ('cb', 'jQuery1123002362902683982293_1593400265041^'),
            #     ('pageindex', '17^'),
            #     ('id', '6018651^'),
            #     ('sort', '1^'),
            #     ('ft', '1^'),
            #     ('code', '601865^'),
            #     ('market', '1^'),
            #     ('_', '1593400265053'),
            # )

            params = (
                ('pagesize', '144'),
                ('ut', '7eea3edcaed734bea9cbfc24409ed989'),
                ('dpt', 'wzfscj'),
                #('cb', 'jQuery1123002362902683982293_1593400265041'),
                ('pageindex', str(page_num)),
                ('id', str(stock_id)),
                ('sort', '1'),
                ('ft', '1'),
                ('code', str(stock_code)),
                ('market', str(market_id)),
                ('_', '1593400265053'),
             )


            #print(params)
            session = PACSession()
            response = session.get('http://push2ex.eastmoney.com/getStockFenShi', headers=headers, params=params, cookies=cookies, verify=False)
            #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi',params=params)
            #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi?pagesize=144&ut=7eea3edcaed734bea9cbfc24409ed989&dpt=wzfscj&pageindex=7&id=6018651&sort=1&ft=1&code=601865&market=1&_=1593400265053')

            #NB. Original query string below. It seems impossible to parse and
            #reproduce query strings 100% accurately so the one below is given
            #in case the reproduced version is not "correct".
            #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi?pagesize=144^&ut=7eea3edcaed734bea9cbfc24409ed989^&dpt=wzfscj^&pageindex=10^&id=6018651^&sort=1^&ft=1^&code=601865^&market=1^&_=1593400265053')
            #response.text

            dict = json.loads(response.text)['data']['data']
            #stock_name = json.loads(response.text)['data']['n']
            df = json_normalize(dict)
            #df.tail(10)
            timelst = pd.to_datetime(df['t'],format='%H%M%S')
            #timelst.dt.time
            df['t'] = timelst.dt.time
            df['p'] = df['p']/1000
            
            #csv_file_name = "../Stock_data/fenshi/" + "fenshi_" + stock_code + "_" + stock_name + "_" + str(curr_time.date()) + ".csv"

            if page_num == 0:
                df.to_csv(csv_file_name, index=False, encoding="utf_8_sig")
            else:
                df.to_csv(csv_file_name, index=False, header=False, mode='a', encoding="utf_8_sig")
            #print("Page " + str(page_num) + ' stored!')

        except:
            break
            
    filed_num+= 1
    #print("\r" + str(filed_num) + ": " + stock_code + " " + stock_name + " fenshi stored!", end="")
    print(str(filed_num) + ": " + stock_code + " " + stock_name + " fenshi stored!")

print("WELL DONE!!!")

1: 300052 中青宝 fenshi stored!
2: 688023 安恒信息 fenshi stored!
3: 000951 中国重汽 fenshi stored!
4: 300741 华宝股份 fenshi stored!
5: 603822 嘉澳环保 fenshi stored!
6: 002777 久远银海 fenshi stored!
7: 300280 紫天科技 fenshi stored!
8: 300161 华中数控 fenshi stored!
9: 603897 长城科技 fenshi stored!
10: 300212 易华录 fenshi stored!
11: 002983 芯瑞达 fenshi stored!
12: 603915 国茂股份 fenshi stored!
13: 002126 银轮股份 fenshi stored!
14: 600999 招商证券 fenshi stored!
15: 002456 欧菲光 fenshi stored!
16: 002475 立讯精密 fenshi stored!
17: 603021 山东华鹏 fenshi stored!
18: 300454 深信服 fenshi stored!
19: 002152 广电运通 fenshi stored!
20: 603818 曲美家居 fenshi stored!
WELL DONE!!!


In [ ]:
t: 时间，p：元*1000，v:手数 bs: 1代表卖方主动(绿色)；2代表买方主动（红色）

# 可以下载全的版本（2020701午）

In [3]:
import os
import requests
from pypac import PACSession
import pandas as pd
import json
from pandas import json_normalize
import datetime

curr_time = datetime.datetime.now()
hq_csv =  "../Stock_data/hangqing/" + "hangqing_A_" + str(curr_time.date()) + ".csv"
df1 = pd.read_csv(hq_csv,dtype={'f12':str})

file_dir = "../Stock_data/fenshi/"

exit_flag = 0

while True:
    
    for root, dirs, files in os.walk(file_dir):
        namelist = []
        for name in files:
            #print(os.path.join(root, name))
            #print(name[7:13])
            namelist.append(name[7:13])

    df1 = df1[~df1['f12'].isin(namelist)]
    
    if len(df1['f12']) < 1:
        break
    
    if exit_flag>5:
        df1.to_csv("../Stock_data/hangqing/na_stock_list_" + str(curr_time.date()) + ".csv")
        exit()
    
    #df1[['f12','f14', 'f13']].head(10)
    #df1.set_index('f12', inplace=True)
    #df1.loc['300820', 'f13']
    #df1.index[:10]

    filed_num = len(namelist)

    for stock_code in df1['f12']:  

        #market_id = df1.loc[stock_code, 'f13'] #上证是1，深证是0
        market_id_se = df1[df1['f12'] == stock_code]['f13']
        market_id = market_id_se.iloc[0]

        if str(market_id) == '0':
            stock_id = stock_code + '2' #上证是1，深证是2
        else:
            stock_id = stock_code + '1' #上证是1，深证是2

        #stock_name = df1.loc[stock_code, 'f14']
        stock_name_se = df1[df1['f12'] == stock_code]['f14']
        stock_name = stock_name_se.iloc[0]
        if stock_name[0] == "*":
            stock_name = stock_name[1:]

        csv_file_name = "../Stock_data/fenshi/" + "fenshi_" + stock_code + "_" + stock_name + "_" + str(curr_time.date()) + ".csv"

        for page_num in range(50):
            try:
                cookies = {
                    'em_hq_fls': 'js',
                    'pgv_pvi': '9744040960',
                    '_qddaz': 'QD.efn5wx.2y13rz.kbiype8t',
                    'intellpositionL': '720.95px',
                    'cowminicookie': 'true',
                    'emshistory': '^%^5B^%^22600519^%^22^%^5D',
                    'intellpositionT': '755px',
                    'qgqp_b_id': '3b5494631b0d78184891d0de53c4b3d9',
                    'st_si': '81244739108722',
                    'st_asi': 'delete',
                    '_sm_au_c': 'kLQABAOhavxwaGMtlop7idfc5VbaufLUcxaJVNQw0aBAgAAAAXYqAxNyvGhuw4qcGTDpRh0vz6imEFA2Tbt8n3TUV7ks=',
                    'st_pvi': '20795103616993',
                    'st_sp': '2020-06-17^%^2014^%^3A02^%^3A47',
                    'st_inirUrl': 'https^%^3A^%^2F^%^2Fcn.bing.com^%^2F',
                    'st_sn': '26',
                    'st_psi': '20200629111105268-0-8277370083',
                    'HAList': 'a-sh-601865-^%^u798F^%^u83B1^%^u7279^%^2Ca-sz-002813-^%^u8DEF^%^u7545^%^u79D1^%^u6280^%^2Ca-sh-600956-N^%^u65B0^%^u5929^%^2Ca-sh-600519-^%^u8D35^%^u5DDE^%^u8305^%^u53F0^%^2Ca-sz-000858-^%^u4E94^%^20^%^u7CAE^%^20^%^u6DB2^%^2Ca-sz-002129-^%^u4E2D^%^u73AF^%^u80A1^%^u4EFD^%^2Ca-sh-603365-^%^u6C34^%^u661F^%^u5BB6^%^u7EBA^%^2Ca-sh-600918-^%^u4E2D^%^u6CF0^%^u8BC1^%^u5238',
                }

                headers = {
                    'Proxy-Connection': 'keep-alive',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
                    'Accept': '*/*',
                    'Referer': 'http://quote.eastmoney.com/f1.html?code=601865^&market=1',
                    'Accept-Language': 'en-US,en;q=0.9',
                }

                # params = (
                #     ('pagesize', '144^'),
                #     ('ut', '7eea3edcaed734bea9cbfc24409ed989^'),
                #     ('dpt', 'wzfscj^'),
                #     ('cb', 'jQuery1123002362902683982293_1593400265041^'),
                #     ('pageindex', '17^'),
                #     ('id', '6018651^'),
                #     ('sort', '1^'),
                #     ('ft', '1^'),
                #     ('code', '601865^'),
                #     ('market', '1^'),
                #     ('_', '1593400265053'),
                # )

                params = (
                    ('pagesize', '144'),
                    ('ut', '7eea3edcaed734bea9cbfc24409ed989'),
                    ('dpt', 'wzfscj'),
                    #('cb', 'jQuery1123002362902683982293_1593400265041'),
                    ('pageindex', str(page_num)),
                    ('id', str(stock_id)),
                    ('sort', '1'),
                    ('ft', '1'),
                    ('code', str(stock_code)),
                    ('market', str(market_id)),
                    ('_', '1593400265053'),
                 )


                #print(params)
                session = PACSession()
                response = session.get('http://push2ex.eastmoney.com/getStockFenShi', headers=headers, params=params, cookies=cookies, verify=False)
                #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi',params=params)
                #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi?pagesize=144&ut=7eea3edcaed734bea9cbfc24409ed989&dpt=wzfscj&pageindex=7&id=6018651&sort=1&ft=1&code=601865&market=1&_=1593400265053')

                #NB. Original query string below. It seems impossible to parse and
                #reproduce query strings 100% accurately so the one below is given
                #in case the reproduced version is not "correct".
                #response = requests.get('http://push2ex.eastmoney.com/getStockFenShi?pagesize=144^&ut=7eea3edcaed734bea9cbfc24409ed989^&dpt=wzfscj^&pageindex=10^&id=6018651^&sort=1^&ft=1^&code=601865^&market=1^&_=1593400265053')
                #response.text

                dict = json.loads(response.text)['data']['data']
                #stock_name = json.loads(response.text)['data']['n']
                df = json_normalize(dict)
                #df.tail(10)
                timelst = pd.to_datetime(df['t'],format='%H%M%S')
                #timelst.dt.time
                df['t'] = timelst.dt.time
                df['p'] = df['p']/1000

                #csv_file_name = "../Stock_data/fenshi/" + "fenshi_" + stock_code + "_" + stock_name + "_" + str(curr_time.date()) + ".csv"

                if page_num == 0:
                    df.to_csv(csv_file_name, index=False, encoding="utf_8_sig")
                else:
                    df.to_csv(csv_file_name, index=False, header=False, mode='a', encoding="utf_8_sig")
                #print("Page " + str(page_num) + ' stored!')

            except:
                break

        filed_num+= 1
        print("\r" + str(filed_num) + ": " + stock_code + " " + stock_name + " fenshi stored!", end="")
        #print(str(filed_num) + ": " + stock_code + " " + stock_name + " fenshi stored!")
    
    exit_flag+= 1
    
    print('\nexit_flag = ' + str(exit_flag))
    #print("\nGOOD!")
    
print("WELL DONE!!")

180: 600817 ST宏盛 fenshi stored!!
exit_flag = 1
WELL DONE!!


# 扫描数据目录中的文件名，将已有的股票从df1中去掉

In [2]:
import requests
from pypac import PACSession
import pandas as pd
import json
from pandas import json_normalize
import datetime

import os

file_dir = "../Stock_data/fenshi/"
for root, dirs, files in os.walk(file_dir):
    namelist = []
    for name in files:
        #print(os.path.join(root, name))
        #print(name[7:13])
        namelist.append(name[7:13])

curr_time = datetime.datetime.now()
#hq_csv =  "../Stock_data/hangqing/" + "hangqing_A_" + str(curr_time.date()) + ".csv" #hangqing_A_2020-06-30
hq_csv =  "../Stock_data/hangqing/" + "hangqing_A_" + "2020-07-01" + ".csv"
df1 = pd.read_csv(hq_csv,dtype={'f12':str})
#df1.set_index('f12', inplace=True)

df2 = df1[~df1['f12'].isin(namelist)] #从df1中去掉某些行，这些行中的'f12'列中包含namelist中的元素

#df1['f12'].isin(namelist).to_csv('111111.csv')
                
#df1['f12']
# stock_code = '688196'
# market_id = df1[df1['f12'] == stock_code]['f13']

#market_id = df1.loc[stock_code, 'f13']

# market_id.iloc[0]
#df2.to_csv("nan_stock_list1.csv")
print(len(df1['f12']))
# print(len(df2))
print(len(namelist))
# namelist
df2['f12']

180
0


0      688528
1      300846
2      688520
3      688278
4      002069
        ...  
175    600021
176    603896
177    300639
178    002467
179    600817
Name: f12, Length: 180, dtype: object